## 🤔 Why Do We Need Chunking?

**Problem:** AI models have limited "context windows"

```
┌─────────────────────────────────────────────────────────┐
│ Your PDF: 50 pages × 3000 chars = 150,000 characters   │
│                                                         │
│ AI Context Window: ~4,000 - 8,000 characters           │
│                                                         │
│ Problem: Can't fit entire document! ❌                  │
└─────────────────────────────────────────────────────────┘
```

**Solution:** Split into small chunks, find the relevant ones, send only those!

```
Full Document  →  Split into Chunks  →  Find Relevant  →  Send to AI
(too big)          (500 chars each)      (2-3 chunks)     (fits!)
```

## Step 1: Load Documents (from Notebook 1)

In [ ]:
# First, let's load our documents
from langchain_community.document_loaders import PyPDFLoader
import os

# Set up paths
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
data_folder = os.path.join(project_root, 'data')

# Load all PDFs
all_pages = []
pdf_files = [f for f in os.listdir(data_folder) if f.endswith('.pdf')]

print("📚 Loading PDFs...")
for pdf_name in pdf_files:
    loader = PyPDFLoader(os.path.join(data_folder, pdf_name))
    pages = loader.load()
    all_pages.extend(pages)
    print(f"   ✅ {pdf_name}: {len(pages)} pages")

print(f"\n📄 Total pages: {len(all_pages)}")

---

## Step 2: Understand the Problem

In [ ]:
# Let's see how big our pages are
print("📊 Page Size Analysis:")
print("="*60)

page_lengths = [len(page.page_content) for page in all_pages]

print(f"Shortest page: {min(page_lengths):,} characters")
print(f"Longest page:  {max(page_lengths):,} characters")
print(f"Average page:  {sum(page_lengths)//len(page_lengths):,} characters")
print(f"Total content: {sum(page_lengths):,} characters")

print("\n⚠️ Problem: Most AI models can only handle 4,000-8,000 characters at once!")

---

## Step 3: Introduction to Text Splitting

LangChain provides several text splitters:

| Splitter | Description | Best For |
|----------|-------------|----------|
| CharacterTextSplitter | Splits by character count | Simple cases |
| **RecursiveCharacterTextSplitter** | Splits smartly at sentence/paragraph boundaries | **Most cases** ✓ |
| TokenTextSplitter | Splits by token count | When using specific models |
| MarkdownTextSplitter | Respects markdown structure | Markdown documents |

In [ ]:
# Import the recommended text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter

print("✅ RecursiveCharacterTextSplitter imported!")
print("\n📖 Why 'Recursive'?")
print("   It tries to split at natural boundaries:")
print("   1. First, try splitting at paragraphs (\\n\\n)")
print("   2. If still too big, try sentences (.)")
print("   3. If still too big, try words (space)")
print("   4. Last resort: split at characters")

---

## Step 4: Configure the Text Splitter

In [ ]:
# Create a text splitter with specific settings
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,      # Target size of each chunk
    chunk_overlap=50,    # Overlap between chunks
    length_function=len, # How to measure length
    separators=["\n\n", "\n", ". ", " ", ""]  # Split priorities
)

print("✅ Text splitter configured!")
print(f"\n📐 Settings:")
print(f"   Chunk size: 500 characters")
print(f"   Overlap: 50 characters")

### 💡 Understanding the Parameters

**chunk_size** (500 characters)
- How big each chunk should be
- Smaller = more precise retrieval, but less context
- Larger = more context, but might include irrelevant info

**chunk_overlap** (50 characters)
- How much chunks should overlap
- Prevents cutting sentences in half

```
Chunk 1: [........text content.......][overlap]
Chunk 2:                       [overlap][........text content.......]
                                   ↑
                           Shared content!
```

---

## Step 5: Split the Documents

In [ ]:
# Split all pages into chunks
print("✂️ Splitting documents into chunks...\n")

chunks = text_splitter.split_documents(all_pages)

print(f"📄 Original pages: {len(all_pages)}")
print(f"📦 Chunks created: {len(chunks)}")
print(f"\n📊 Ratio: {len(chunks)/len(all_pages):.1f} chunks per page (average)")

### 💡 Explanation

```
Before:  25 pages  (each ~2000-3000 characters)
                    ↓
After:   ~100 chunks  (each ~500 characters)
```

Each page gets split into multiple smaller chunks.

---

## Step 6: Examine the Chunks

In [ ]:
# Look at chunk statistics
print("📊 Chunk Size Analysis:")
print("="*60)

chunk_lengths = [len(chunk.page_content) for chunk in chunks]

print(f"Smallest chunk: {min(chunk_lengths)} characters")
print(f"Largest chunk:  {max(chunk_lengths)} characters")
print(f"Average chunk:  {sum(chunk_lengths)//len(chunk_lengths)} characters")

# Show distribution
print("\n📈 Size Distribution:")
ranges = [(0, 200), (200, 400), (400, 500), (500, 600)]
for low, high in ranges:
    count = sum(1 for l in chunk_lengths if low <= l < high)
    bar = "█" * (count // 2)
    print(f"   {low:3}-{high:3}: {bar} ({count})")

In [ ]:
# Look at a sample chunk
print("📝 Sample Chunk (first one):")
print("="*60)
print(chunks[0].page_content)
print("="*60)
print(f"\n📋 Metadata: {chunks[0].metadata}")
print(f"📏 Length: {len(chunks[0].page_content)} characters")

---

## Step 7: Visualize the Overlap

In [ ]:
# Find consecutive chunks from the same page to show overlap
print("🔍 Finding overlap between consecutive chunks...\n")

# Get chunks from the same page
page_1_chunks = [c for c in chunks if c.metadata.get('page', -1) == 1]

if len(page_1_chunks) >= 2:
    chunk_a = page_1_chunks[0]
    chunk_b = page_1_chunks[1]
    
    # Show end of first chunk
    print("📄 End of Chunk 1:")
    print("-"*40)
    print(f"...{chunk_a.page_content[-100:]}")
    
    # Show start of second chunk
    print("\n📄 Start of Chunk 2:")
    print("-"*40)
    print(f"{chunk_b.page_content[:100]}...")
    
    # Find the overlap
    end_text = chunk_a.page_content[-50:]
    if end_text in chunk_b.page_content:
        print("\n✅ Overlap detected! The end of Chunk 1 appears at the start of Chunk 2.")
else:
    print("Not enough chunks from same page to demonstrate overlap.")

---

## 🧪 Experiment: Different Chunk Sizes

In [ ]:
# Compare different chunk sizes
print("🧪 Experimenting with different chunk sizes:")
print("="*60)

sizes_to_try = [200, 500, 1000, 2000]

for size in sizes_to_try:
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=size,
        chunk_overlap=size // 10  # 10% overlap
    )
    test_chunks = splitter.split_documents(all_pages)
    
    print(f"\n   Chunk size {size}:")
    print(f"   └── {len(test_chunks)} chunks created")
    print(f"       (avg: {sum(len(c.page_content) for c in test_chunks)//len(test_chunks)} chars)")

### 💡 Choosing Chunk Size

| Size | Chunks | Pros | Cons |
|------|--------|------|------|
| **200** | Many | Very precise retrieval | Less context per chunk |
| **500** | Medium | Good balance ✓ | Good for most cases |
| **1000** | Fewer | More context | Might include irrelevant text |
| **2000** | Few | Lots of context | Harder to find specific info |

**Recommendation:** Start with 500 and adjust based on your results.

---

## ✅ Summary

In this notebook, you learned:

1. **Why chunking is needed** - AI models have limited context windows
2. **RecursiveCharacterTextSplitter** - Splits at natural boundaries
3. **Chunk size** - Balance between precision and context (~500 chars)
4. **Chunk overlap** - Prevents cutting sentences in half (~50 chars)

## ➡️ Next Step

In **Notebook 3: Embeddings**, you'll learn how to convert these text chunks into numbers (vectors) that computers can search.

---

**Variables to use in next notebook:**
- `chunks` - List of Document chunks ready for embedding